In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import metrics

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files 
uploaded=files.upload()

Saving twitter.csv to twitter (1).csv


In [ ]:
import io
data= pd.read_csv('twitter.csv')
data

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [ ]:
x=data["tweet"]
y=data["label"]

In [ ]:
y.value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [ ]:
#from nltk.corpus import stopwords
#temp = []
#s = " "
#for i in x:
  #filtered_words = [w.lower() for w in i if ((w.lower()>='a') and (w.lower()<='z')) or (w == ' ')]

  #s = s+str(filtered_words)
  
  #temp.append(s)


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


space_replace=re.compile('[/(){}|@#ð]')
symbol=re.compile('[^-9a-z#]')
stopword=set(stopwords.words('english'))

def clean(x):
   x=x.lower()
   x=space_replace.sub(' ',x)
   x=symbol.sub(' ',x)
   x=' '.join(w for w in x.split() if w not in stopword)
   return x

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
x=x.apply(clean)
x

0        user father dysfunctional selfish drags kids d...
1        user user thanks lyft credit use cause offer w...
2                                           bihday majesty
3                              model love u take u time ur
4                            factsguide society motivation
                               ...                        
31957                                   ate user isz youuu
31958    see nina turner airwaves trying wrap mantle ge...
31959      listening sad songs monday morning otw work sad
31960    user sikh temple vandalised calgary wso condem...
31961                                    thank user follow
Name: tweet, Length: 31962, dtype: object

In [ ]:
max_words=2000
max_seq_len=200
embedding_dim=50


In [ ]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x.values)
word_id=tokenizer.word_index

word_id

In [ ]:
x= tokenizer.texts_to_sequences(x.values)
x

In [ ]:
x=pad_sequences(x,maxlen=max_seq_len)
x

array([[   0,    0,    0, ...,   22,  161,  345],
       [   0,    0,    0, ...,  332,  525, 1372],
       [   0,    0,    0, ...,    0,    0,   16],
       ...,
       [   0,    0,    0, ...,   47,   25,   57],
       [   0,    0,    0, ..., 1457, 1458,  552],
       [   0,    0,    0, ...,   91,    1,   73]], dtype=int32)

In [ ]:
y.values

from pandas import get_dummies
y_=get_dummies(y)
y=y_.values
y

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split 
x_train,_x_test,y_train,_y_test=train_test_split(x,y,shuffle=True)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()

model.add(Embedding(max_words, embedding_dim, input_length=x.shape[1]))
model.add(LSTM(100))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 50)           100000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
Total params: 165,552
Trainable params: 165,552
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, validation_split=0.1, epochs = 5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 21573 samples, validate on 2398 samples
Epoch 1/5
21573/21573 [==============================] - 359s 17ms/step - loss: 0.1776 - accuracy: 0.9445 - val_loss: 0.1442 - val_accuracy: 0.9491
Epoch 2/5
21573/21573 [==============================] - 355s 16ms/step - loss: 0.1146 - accuracy: 0.9593 - val_loss: 0.1388 - val_accuracy: 0.9545
Epoch 3/5
 5184/21573 [======>.......................] - ETA: 4:27 - loss: 0.1086 - accuracy: 0.9576

KeyboardInterrupt: ignored

In [ ]:
#model.save('weight_rnn_twitter.h5')

In [ ]:
#from keras.models import load_model
#model=load_model('weight_rnn_twitter.h5')

In [ ]:

model.evaluate(_x_test, _y_test)

7991/7991 [==============================] - 12s 1ms/step


[0.15560717770030055, 0.9516956806182861]

In [ ]:

s= "when looting starts shooting starts"
print (s)

when looting starts shooting starts


In [ ]:
s=clean(s)
s

'looting starts shooting starts'

In [ ]:
s= tokenizer.texts_to_sequences(s)
s

AttributeError: ignored

In [ ]:
s=pad_sequences(x,maxlen=max_seq_len)
s

array([[   0,    0,    0, ...,   22,  161,  345],
       [   0,    0,    0, ...,  332,  525, 1372],
       [   0,    0,    0, ...,    0,    0,   16],
       ...,
       [   0,    0,    0, ...,   47,   25,   57],
       [   0,    0,    0, ..., 1457, 1458,  552],
       [   0,    0,    0, ...,   91,    1,   73]], dtype=int32)

In [ ]:
model.predict(s)

array([[9.95822072e-01, 4.17788513e-03],
       [7.36113966e-01, 2.63886094e-01],
       [9.98840392e-01, 1.15968543e-03],
       ...,
       [9.99897242e-01, 1.02795435e-04],
       [3.82256391e-03, 9.96177435e-01],
       [9.48392212e-01, 5.16077466e-02]], dtype=float32)